In [1]:
%matplotlib ipympl
import os.path as op
from mayavi import mlab
import pickle
import numpy as np
import matplotlib.pyplot as plt
import mne

********************************************************************************
         to build the TVTK classes (8.2). This may cause problems.
         Please rebuild TVTK.
********************************************************************************



In [9]:
## Set parameters of interest here##
############################################
task = 'fixed_gaze'
conditions = ['armenian', 'phase_rand']
noise_cov= 'merged_nc'
method = 'dSPM'
tmin = 0.0
tmax = 0.450
p_threshold = 0.05
n_permutations = 1000
###########################################
data_folder = op.join('D:\\', 'Zsuzsa', 'source_localization')
task_dir = op.join(data_folder, task)
time_label = str(int(tmin * 1000)) + '-' + str(int(tmax * 1000)) + 'ms'
out_folder = op.join(task_dir, 'results', noise_cov, method, 'clusters')
stc_result = op.join(out_folder, conditions[0] + '_' + conditions[1] + '_' + time_label + '_p-' + str(p_threshold) + '_nperm-' + str(int(n_permutations)))
stc = mne.read_source_estimate(stc_result, 'fsaverage')

## View cluster with p < 0.05 without time-course

In [10]:
views =['lateral', 'caudal', 'ventral']
tview_kwargs = dict(hemi='split',views=views, #clim=dict(kind='value', pos_lims=[18.0, 20.0, 25.0]),
                    time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, time_viewer = True)
brain = stc.plot(**tview_kwargs)

Using control points [ 50.  64. 198.]


## Pickled cluster object - view cluster time course

In [6]:
result_file = op.join(out_folder, conditions[0] + '_' + conditions[1] + '_' + time_label + '_nperm-' + str(int(n_permutations))+'_all_clu.pkl')
clu = pickle.load( open( result_file, "rb" ) )
T_obs, clusters, cluster_p_values, H0 = clu
n_times, n_vertices = T_obs.shape
data = np.zeros((n_vertices, n_times))
good_cluster_inds = np.where(cluster_p_values < 0.05)[0]
print(good_cluster_inds)

[ 908  943 1473 3490 3508]


In [7]:

v_inds = clusters[good_cluster_inds[0]][1]
t_inds = clusters[good_cluster_inds[0]][0]
data[v_inds, t_inds] = T_obs[t_inds, v_inds]

In [8]:
stc.data = data
views =['lateral', 'caudal', 'ventral']
tview_kwargs = dict(hemi='split',views=views, clim=dict(kind='value', pos_lims=[1.0, 2.5, 7.0]),
                    time_unit='s', smoothing_steps=5, colorbar = True, size = 1400, time_viewer = True)
brain = stc.plot(**tview_kwargs)